# Configuration

In [35]:
nb_users = 60
resource_capacity = 80
sim_duration = 360 # 1 = 1 mn

In [36]:
#debug
#nb_users = 1
#resource_capacity = 80
#sim_duration = 180 # 1 = 1 mn

In [37]:
session_configuration = """\
session:
    name: nominal test
    scenarios:
        - name: Sc1
          weight: 2
          steps:
                loop:
                    tasks:
                        - name: step 1
                          duration: 10
                          resources : 50   
                          wait: 30
        - name: Sc2
          weight: 10
          steps:
                init:
                    tasks:
                        - name: load data
                          duration: 5
                          resources : 3
                          parallel: 10
                        - name: verify data
                          duration: 10
                loop:
                    repeat: 2
                    tasks:
                        - name: step 1-3
                          duration: 2
                          resources : 5   
                          wait: 5
                          repeat: 3
                        - name: step 4
                          duration: 5
                          resources : 10      
                          wait: 30
                finally:
                    tasks:
                        - name: whole step
                          duration: 3
                          resources : 7
                          repeat: 4       

"""



# Functions

In [38]:
!pip install simpy plotly

### Refactoring Goal
- randomization of tasks duration
- structure of user scenario


In [39]:
import simpy
import datetime
import pandas as pd
import plotly.express as px
import logging
from enum import Enum

import random
from itertools import repeat

# new
from ruamel.yaml import YAML


In [40]:
log_filename = "logs-10.log"
mainLogger = logging.getLogger()
mainLogger = logging.getLogger()
fhandler = logging.FileHandler(filename=log_filename, mode='w')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
mainLogger.addHandler(fhandler)
mainLogger.setLevel(logging.DEBUG)
mainLogger.debug("test")

In [41]:
class Metric(Enum):
    RW = "Requests Waiting"
    BS = "Busy Slots"
    AU = "Active Users"

In [42]:
class Scenario1_old:
    def __init__(self):
        self.name = "Sc1"
        self.tasks = []
        self.tasks.append( 
            { 'Name': "Process-1",
              'Duration': 5,
              'Res': 10
            })        
        self.tasks.append( 
            { 'Name': "Verify-1",
              'Duration': 2
            })

class Scenario2_old:
    def __init__(self):
        self.name = "Sc2"
        self.tasks = []
        self.tasks.append( 
            { 'Name': "Process-2.1",
              'Duration': 2,
              'Res': 5
            })        
        self.tasks.append( 
            { 'Name': "Verify-2.1",
              'Duration': 1
            })
        self.tasks.append( 
            { 'Name': "Wait-2.1",
              'Duration': 1
            })
        self.tasks.append( 
            { 'Name': "Process-2.2",
              'Duration': 3,
              'Res': 5
            })        
        self.tasks.append( 
            { 'Name': "Verify-2.2",
              'Duration': 1
            })
        
class Scenario3_old:
    def __init__(self):
        self.name = "Sc3"
        self.tasks = []
        self.tasks.append( 
            { 'Name': "Process-1",
              'Duration': 8,
              'Res': 40
            })        
        self.tasks.append( 
            { 'Name': "Verify-1",
              'Duration': 2
            })



In [43]:
class User:
    def __init__(self, id, scenario, world):
        self.id = id
        self.scenario = scenario
        self._world = world
        self.taskid = 0 # new
        self.create()
        # Start the run process everytime an instance is created.
        # create itself as a processs
        self.action = self._world.env.process(self.run())

    def create(self):
        self.enteringAt = self._world.env.now
        self.name = "User-%03d" % self.id
        mainLogger.info(f"user created {self.name}")
        self._world.user_monitor.report_new_user(self)

    def run_old(self):
        while True:
            self.taskid += 1
            for task in self.scenario.tasks:
                taskname = task['Name']
                task_duration = task['Duration']
                mark = self._world.env.now
                mainLogger.debug(f"{self.name} starts task {taskname} at %d" % mark)

                if 'Res' in task:
                    self._world.user_monitor.report_start(
                            self.name, 
                            self.scenario, #new
                            taskname, 
                            self.taskid)
                    # We yield the process that process() returns
                    # to wait for it to finish
                    amount = task['Res']
                    yield self._world.env.process(self.process_task(task_duration, amount))
                    #yield self._world.env.process(self.process_task(task_duration))
                    self._world.user_monitor.report_stop(
                            self.name, 
                            self.scenario, #new
                            taskname, 
                            self.taskid)
                else:
                    # wait some time even if no tracked
                    yield self._world.env.timeout(task_duration)

                mainLogger.debug(f"{self.name} ends task {taskname} at %d" % mark)

    def run(self):
        scenario = self.scenario
        mainLogger.debug(f"entering scenario: {scenario['name']}")
        mainLogger.debug(f"steps: {scenario['steps']}")
        if 'init' in scenario['steps']:
            mainLogger.debug("has init") 
            mainLogger.debug("run_step_tasks init")
            process = self.run_step_tasks(scenario['steps']['init']['tasks'])
            yield self._world.env.process(process)             

        if 'loop' in scenario['steps']:
            mainLogger.debug("has loop")
            step_loop = scenario['steps']['loop']
            if 'repeat' in step_loop:
                counter = 0
                while counter < step_loop['repeat']:
                    mainLogger.debug("run_step_tasks loop")
                    process = self.run_step_tasks(scenario['steps']['loop']['tasks'])
                    yield self._world.env.process(process)             
                    counter += 1
            else:
                mainLogger.debug("run_step_tasks loop infinite")
                process = self.run_step_tasks(scenario['steps']['loop']['tasks'])
                yield self._world.env.process(process)             

        if 'finally' in scenario['steps']:
            mainLogger.debug("has finally")
            mainLogger.debug("run_step_tasks finally")
            process = self.run_step_tasks(scenario['steps']['finally']['tasks'])
            yield self._world.env.process(process)             

    def run_step_tasks(self, tasks):
        mainLogger.debug(f"entering run_step_tasks {tasks}")
        for task in tasks:
            mainLogger.debug(f"run_step_tasks::task: {task}")
            yield self._world.env.process(self.run_task(task))

    def run_task(self, task):
        mainLogger.debug(f"entering run_task {task} id:{self.taskid}")
        max_count = 1
        if 'repeat' in task:
            max_count = task['repeat']
        counter = 0
        while counter < max_count:
            self.taskid += 1
            mainLogger.debug(f"run task {task['name']} for {task['duration']}")
            if 'resources' in task:
                res_amount = task['resources']
                if 'parallel' in task:
                    mainLogger.debug(f"run_task::in parallel")
                    res_amount = res_amount * task['parallel']
                mainLogger.debug(f"task resources amount {res_amount}")
                self._world.user_monitor.report_start(
                        self.name, 
                        self.scenario['name'],
                        task['name'], 
                        self.taskid)
                process = self.process_task(task['duration'], res_amount)
                yield self._world.env.process(process)
                self._world.user_monitor.report_stop(
                        self.name, 
                        self.scenario['name'], 
                        task['name'], 
                        self.taskid)
                mainLogger.debug(f"task processing completed")
            else:
                mainLogger.debug(f"wait after task for {task['duration']}")
                yield self._world.env.timeout(task['duration'])
                mainLogger.debug(f"wait after task completed")
                            
            if 'wait' in task:
                mainLogger.debug(f"manual task for {task['wait']}")
                yield self._world.env.timeout(task['wait'])
                mainLogger.debug(f"manual task completed")
            
            # increment counter                    
            counter += 1
                  
    def process_task(self, duration, amount):
        mainLogger.debug(f"entering process task at %d" % self._world.env.now)
        with Job(self._world.res,amount) as req:
            yield req
            yield self._world.env.timeout(duration)
        mainLogger.debug(f"exiting process task at %d" % self._world.env.now)



In [44]:
class Clock:
    def __init__(self):
        self.base_epoch = datetime.datetime.now().timestamp()
        mainLogger.info(f"Clock created - base {self.base_epoch}")

    def to_date(self, tick):
        epoch_time = self.base_epoch + tick*60 #mn
        datetime_time = datetime.datetime.fromtimestamp(epoch_time)
        return datetime_time

In [45]:
class UsersMonitor:
    def __init__(self, world):
        self._world = world # new
        # init parameters are self reported
        # start and stop events
        self.start_data = []
        self.stop_data = []
        # list of users
        self.users = []
        
    def report_new_user(self, user):
        self.users.append(user)            
        
    #def report_start(self, username, taskname, taskid):
    def report_start(self, username, scenarioname, taskname, taskid):
        mark = self._world.env.now
        self.start_data.append(
            dict(  
                StartMark=mark,
                Start=world.clock.to_date(mark),
                Username=username,
                Scenario=scenarioname, # new
                Task=taskname,
                TaskId=taskid
            )
        )       
        
    #def report_stop(self, username, taskname, taskid):
    def report_stop(self, username, scenarioname, taskname, taskid):
        mark = self._world.env.now
        self.stop_data.append(
            dict(  
                FinishMark=mark,
                Finish=world.clock.to_date(mark),
                Username=username,
                Scenario=scenarioname, # new
                Task=taskname,
                TaskId=taskid
            )
        )
        
                
    def collect(self):
        df_start = pd.DataFrame(self.start_data)
        df_stop = pd.DataFrame(self.stop_data)
        df = pd.merge(df_start, df_stop, how='left', 
                      on = ['Username', 'Scenario', 'Task', 'TaskId'])
        df['Duration'] = df['FinishMark'] - df['StartMark']
        return df


In [46]:
# wake up every tick and collect
class UsersGenerator:
    def __init__(self, world, max_nb_users=10):
        self._world = world
        self._max_nb_users = max_nb_users
        mainLogger.info("creating user generator for %s users", self._max_nb_users)
        self.data = []
        self.active_users = []
        self.user_count = 0
        # this will be used as a process
        self.action = world.env.process(self.run())
        # new - moves in world 
        #random.shuffle(self._world.scenarios)

    def run(self):
        while True:

            if self.user_count < self._max_nb_users:
                self.create_user()
        
            self.report()

            tick_duration = 1
            yield self._world.env.timeout(tick_duration)

    def create_user(self):

        # new - replaced method
        #scenario = random.choice(self._world.scenarios)
        # never run rare scenarios
        
        # count start at base 0
        #i_scenario = self.user_count % len(scenarios)
        #scenario = self._world.scenarios[i_scenario]
 
        #new - first get scenario index in randomized list 
        # count start at base 0
        #i_scenario = self.user_count % len(scenarios)
        #scenario = self._world.scenarios[i_scenario]

        i_scenario_index = self.user_count % len(self._world.scenarios_index)
        i_scenario = self._world.scenarios_index[i_scenario_index]
        scenario = self._world.scenarios[i_scenario]
    
        # first user is labelled -001
        self.user_count += 1
        user = User(self.user_count, 
                    scenario,
                    self._world)
        self.active_users.append(user)
        mark = self._world.env.now
        mainLogger.debug(f"{len(self.active_users)} active users at %d" % mark)
        
    def report(self):
        mark = self._world.env.now
        active_users_count = len(self.active_users)

        self.data.append(
            dict(  
                Mark=mark,
                Timestamp=self._world.clock.to_date(mark),
                Metric=Metric.AU.value,
                Value=active_users_count
            )
        )       
        
    
    def collect(self):
        return pd.DataFrame(self.data)


In [47]:
class Job:
    def __init__(self, res, items=1):
        self.res = res
        self.items = items
        mainLogger.debug(f"creating job with amount {self.items}")
        
    def __enter__(self):
        mainLogger.debug("__enter__" )
        return self.res.get(self.items).__enter__()

    def __exit__(self, exc_type, exc_val, exc_tb):
        mainLogger.debug("__exit__" )
        mainLogger.debug("exc_type {exc_type} exc_val {exc_val} exc_tb {exc_tb}" )
        self.res.put(self.items).__exit__(exc_type, exc_val, exc_tb)


In [48]:
class SystemResource(simpy.resources.container.Container):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        mainLogger.info(f"create resource with capacity {self.capacity}")
        self.processing_data = []
        self.waiting_data = []
        mainLogger.info(f"initial level {self.level}")

    def get(self, *args, **kwargs):
        amount = args[0]
        mainLogger.debug(f"received request resource - amount {amount} at %d" % self._env.now)
        mainLogger.debug(f"level (available) {self.level} at %d" % self._env.now)
        mainLogger.debug(f"{len(self.get_queue)} waiting at %d" % self._env.now)
        mainLogger.debug(f"{self.used()} processing at %d" % self._env.now)
        self.processing_data.append((self._env.now, self.used()))
        self.waiting_data.append((self._env.now, len(self.get_queue)))
        return super().get(*args, **kwargs)

    def put(self, *args, **kwargs):
        amount = args[0]
        mainLogger.debug(f"received release resource - amount {amount} at %d" % self._env.now)
        mainLogger.debug(f"level (available) {self.level} at %d" % self._env.now)
        mainLogger.debug(f"{len(self.get_queue)} waiting at %d" % self._env.now)
        mainLogger.debug(f"{self.used()} processing at %d" % self._env.now)
        self.processing_data.append((self._env.now, self.used()))
        self.waiting_data.append((self._env.now, len(self.get_queue)))
        return super().put(*args, **kwargs)

    def used(self):
        return self.capacity - self.level

In [49]:
class SystemResource_old(simpy.Resource):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        mainLogger.info(f"create resource with capacity {self.capacity}")

    def request(self, *args, **kwargs):
        mainLogger.debug("request resource at %d" % self._env.now)
        return super().request(*args, **kwargs)

    def release(self, *args, **kwargs):
        mainLogger.debug("release resource at %d" % self._env.now)
        return super().release(*args, **kwargs)


In [50]:
# wake up every tick and collect
class SystemMonitoringAgent:
    def __init__(self, world):
        self._world = world
        mainLogger.info("creating agent")
        self.data = []
        # this will be used as a process
        self.action = world.env.process(self.run())

    def run(self):
        while True:

            mark = self._world.env.now
            #occupied_slots = self._world.res.count
            occupied_slots = self._world.res.used()
            #requests_waiting = len(self._world.res.queue)
            requests_waiting = len(self._world.res.get_queue)
            
            #new
            mainLogger.debug(f"level {self._world.res.level} at %d" % mark)
            mainLogger.debug(f"{occupied_slots} occupied slots at %d" % mark)
            mainLogger.debug(f"{requests_waiting} requests waiting at %d" % mark)

            self.data.append(
                dict(  
                    Mark=mark,
                    Timestamp=self._world.clock.to_date(mark),
                    Metric=Metric.BS.value,
                    Value=occupied_slots
                )
            )       
            self.data.append(
                dict(  
                    Mark=mark,
                    Timestamp=self._world.clock.to_date(mark),
                    Metric=Metric.RW.value,
                    Value=requests_waiting
                )
            ) 
            
            tick_duration = 1
            yield self._world.env.timeout(tick_duration)

        
                
    def collect(self):
        return pd.DataFrame(self.data)


In [51]:
class World:
    
    #def __init__(self, scenarios, nb_users=20, resource_capacity=5):
    def __init__(self,  nb_users=20, resource_capacity=5):
        mainLogger.info(f"creating simulation")
        #self.scenarios = scenarios # new
        self.load_scenarios()
        self.env = simpy.Environment()
        self.clock = Clock()
        self.res = SystemResource(self.env, 
                                  init=resource_capacity, #new
                                  capacity=resource_capacity)
        self.user_monitor = UsersMonitor(self)
        self.user_gen = UsersGenerator(self, max_nb_users=nb_users)
        self.res_agent = SystemMonitoringAgent(self)
        self.load_scenarios()
        
    # new                
    def load_scenarios(self):
        yaml = YAML(typ='safe')   # default, if not specfied, is 'rt' (round-trip)
        session = yaml.load(session_configuration)
        self.session_name = session['session']['name']
        mainLogger.info(f"session name: {self.session_name}")
        
        self.scenarios = session['session']['scenarios']
        self.scenarios_index = []
        for i in range(len(self.scenarios)):
            weight = self.scenarios[i]['weight']
            self.scenarios_index.extend(repeat(i, weight))
        # randomize index
        random.shuffle(self.scenarios_index)
        mainLogger.info(f"scenarios_index: {self.scenarios_index}")
                        
    def start(self, sim_duration = 20):
        mainLogger.info(f"starting simulation")
        self.env.run(until=sim_duration)

## Main 

In [52]:
# new -> remplace par yaml et variables en debut de notebook
world = World(nb_users = nb_users, resource_capacity = resource_capacity)

In [53]:
mainLogger.setLevel(logging.INFO)
world.start(sim_duration = sim_duration) # 1 = 1mn

In [54]:
# debug
if False:
    session_configuration = """\
    session:
        name: nominal test
        scenarios:
            - name: Sc1
              weight: 2
              steps:
                    loop:
                        tasks:
                            - name: step 1 debug
                              duration: 10
                              resources : 3   
                              wait: 5
    """
    world = World(nb_users = 1, resource_capacity = 5 )
    mainLogger.setLevel(logging.DEBUG)
    world.start(sim_duration = 20)


# Time Series

In [55]:
df_system = world.res_agent.collect()
#display(df_system)

## Occupied slots

In [56]:
#df = dfs['occupied_slots']
df = df_system[ df_system.Metric == Metric.BS.value ]
fig = px.bar(df, x='Timestamp', y='Value')
fig.show()

## Requests waiting

In [57]:
#df = dfs['requests_waiting']
df = df_system[ df_system.Metric == Metric.RW.value ]
fig = px.bar(df, x='Timestamp', y='Value')
fig.show()

## Active Users

In [58]:
df_users_active = world.user_gen.collect()
#display(df_users_active)

In [59]:
fig = px.bar(df_users_active, x='Timestamp', y='Value')
fig.show()

# Users activity

In [60]:
# debug
df_start = pd.DataFrame(world.user_monitor.start_data)
df_start_sorted = df_start.sort_values(by=['StartMark', 'Scenario', 'TaskId'])
display(df_start_sorted)

,Scenario,Start,StartMark,Task,TaskId,Username
0,Sc2,2021-09-12 17:26:58.629235,0,load data,1,User-001
1,Sc2,2021-09-12 17:27:58.629235,1,load data,1,User-002
2,Sc2,2021-09-12 17:28:58.629235,2,load data,1,User-003
3,Sc2,2021-09-12 17:29:58.629235,3,load data,1,User-004
4,Sc2,2021-09-12 17:30:58.629235,4,load data,1,User-005
5,Sc2,2021-09-12 17:31:58.629235,5,load data,1,User-006
6,Sc1,2021-09-12 17:32:58.629235,6,step 1,1,User-007
7,Sc2,2021-09-12 17:33:58.629235,7,load data,1,User-008
8,Sc1,2021-09-12 17:34:58.629235,8,step 1,1,User-009
9,Sc2,2021-09-12 17:35:58.629235,9,load data,1,User-010


In [61]:
# debug
#df_stop = pd.DataFrame(world.user_monitor.stop_data)
#display(df_stop)

In [62]:
# debug
#df_stop.shape

## Users chronogram

In [63]:
df_users = world.user_monitor.collect()
display(df_users)

,Scenario,Start,StartMark,Task,TaskId,Username,Finish,FinishMark,Duration
0,Sc2,2021-09-12 17:26:58.629235,0,load data,1,User-001,2021-09-12 17:31:58.629235,5,5
1,Sc2,2021-09-12 17:27:58.629235,1,load data,1,User-002,2021-09-12 17:32:58.629235,6,5
2,Sc2,2021-09-12 17:28:58.629235,2,load data,1,User-003,2021-09-12 17:36:58.629235,10,8
3,Sc2,2021-09-12 17:29:58.629235,3,load data,1,User-004,2021-09-12 17:37:58.629235,11,8
4,Sc2,2021-09-12 17:30:58.629235,4,load data,1,User-005,2021-09-12 17:41:58.629235,15,11
5,Sc2,2021-09-12 17:31:58.629235,5,load data,1,User-006,2021-09-12 17:42:58.629235,16,11
6,Sc1,2021-09-12 17:32:58.629235,6,step 1,1,User-007,2021-09-12 17:51:58.629235,25,19
7,Sc2,2021-09-12 17:33:58.629235,7,load data,1,User-008,2021-09-12 17:47:58.629235,21,14
8,Sc1,2021-09-12 17:34:58.629235,8,step 1,1,User-009,2021-09-12 18:01:58.629235,35,27
9,Sc2,2021-09-12 17:35:58.629235,9,load data,1,User-010,2021-09-12 17:56:58.629235,30,21


In [64]:
#new
#fig = px.timeline(df_users, x_start="Start", x_end="Finish", y="Username")
fig = px.timeline(df_users, x_start="Start", x_end="Finish", y="Username", color="Scenario")
#fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
fig.update_yaxes(categoryorder='category descending')
fig.show()

## Users activity distribution

In [65]:
df_users[["Duration"]].describe()

,Duration
count,660.0000
mean,16.5000
std,28.0381
min,2.0000
25%,3.0000
50%,6.0000
75%,9.0000
max,127.0000


In [66]:
mainLogger.debug(f"max={max}")
fig = px.histogram(df_users, x="Duration", nbins=20)
fig.show()

In [67]:
# when starts waiting - more or less 8 users
# as much as processes - more or less 13 users

# Logs

In [68]:
with open(log_filename) as log:
            print(log.read())

2021-09-12 17:26:58,490 - root - DEBUG - test
2021-09-12 17:26:58,626 - root - INFO - creating simulation
2021-09-12 17:26:58,628 - root - INFO - session name: nominal test
2021-09-12 17:26:58,629 - root - INFO - scenarios_index: [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0]
2021-09-12 17:26:58,629 - root - INFO - Clock created - base 1631460418.629235
2021-09-12 17:26:58,629 - root - INFO - create resource with capacity 80
2021-09-12 17:26:58,629 - root - INFO - initial level 80
2021-09-12 17:26:58,629 - root - INFO - creating user generator for 60 users
2021-09-12 17:26:58,629 - root - INFO - creating agent
2021-09-12 17:26:58,631 - root - INFO - session name: nominal test
2021-09-12 17:26:58,631 - root - INFO - scenarios_index: [1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1]
2021-09-12 17:26:58,635 - root - INFO - starting simulation
2021-09-12 17:26:58,635 - root - INFO - user created User-001
2021-09-12 17:26:58,635 - root - INFO - user created User-002
2021-09-12 17:26:58,636 - root - INFO - user cr